# Yolov7

In [1]:
%cd yolov7

C:\Users\hp\yolov7


In [2]:
import Angles as ag
import detection as de
import counter as co 
import Feedback as fb
import Recognition_UPDATE as rec

In [3]:
#--->importing req libraries
try:
    import torch
    from torchvision import transforms
    import pandas as pd
    import matplotlib.pyplot as plt
    import cv2
    import numpy as np
    import math
    import ctypes
    from utils.datasets import letterbox
    from utils.general import non_max_suppression_kpt
    from utils.plots import output_to_keypoint, plot_skeleton_kpts
    import tkinter as tk
    from tkinter import Message, Text
    import os
    import shutil
    import csv
    from PIL import Image, ImageTk
    import pandas as pd
    import datetime
    import time
    import tkinter.ttk as ttk
    import tkinter.font as font
    from pathlib import Path
    import pyttsx3
    import time
    from NBSapi import NBSapi
    from colorthief import ColorThief
    print("Library Loaded Successfully ..........")
except:
    print("Library not Found ! ")

Library Loaded Successfully ..........


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
def load_model():
    model = torch.load("C:\\Users\\hp\\Downloads\\yolov7-w6-pose.pt", map_location=device)['model']
    # Put in inference mode
    model.float().eval()

    if torch.cuda.is_available():
        # half() turns predictions into float16 tensors
        # which significantly lowers inference time
        model.half().to(device)
    return model

model = load_model()

cuda:0


In [5]:
def run_inference(image):
    # Resize and pad image
    image = letterbox(image, 960, stride=64, auto=True)[0] # shape: (567, 960, 3)
    # Apply transforms
    image = transforms.ToTensor()(image) # torch.Size([3, 567, 960])
    if torch.cuda.is_available():
        image = image.half().to(device)
    # Turn image into batch
    image = image.unsqueeze(0) # torch.Size([1, 3, 567, 960])
    with torch.no_grad():
        output, _ = model(image)
     
    return output, image

In [14]:
def draw_keypoints(output, image, EXS, level,EXSType,tts_g):
    global EXSType_show
    global state 
    global cnt
    global EXS_feedback
    global state_fed
    cntr=[]
    personID=0
    
    output = non_max_suppression_kpt(output, 
                                     0.25, # Confidence Threshold
                                     0.65, # IoU Threshold
                                     nc=model.yaml['nc'], # Number of Classes
                                     nkpt=model.yaml['nkpt'], # Number of Keypoints
                                     kpt_label=True)
    with torch.no_grad():
        output = output_to_keypoint(output)
    
    #change format [b,c,h,w] to [h,w,c] for displaying the image
    nimg = image[0].permute(1, 2, 0) * 255
    nimg = nimg.cpu().numpy().astype(np.uint8)
    nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)
    
    #get dic of a recognized ids 
    ids,rec_dic = rec.face_detect(nimg,rec.name(),tts_g)
    for idx in range(output.shape[0]):
        print(idx)
        print(output[idx, 7:])
        #get only 17 keypoints 
        output2 =output[idx, 7:]
        #link each id to its keypoints
        for id in ids.keys():
            if abs(id-output2[0])<5:
                 personID = ids[id]
                    
        #plot key-points 
        plot_skeleton_kpts(nimg, output[idx, 7:].T, 3)
        
        #apply detection,counter,feedback to only recognized person
        if personID > 0:
            rec_list = rec_dic[personID]
            reel_Angles=ag.findAngle(nimg,output[idx, 7:],personID)
            de.detection (nimg,reel_Angles,personID,EXS, level, EXSType,EXSType_show,tts_g,rec_list[0],rec_list[1])
            fb.feedback(nimg,reel_Angles,personID, level,EXS,EXSType_show,EXS_feedback,state_fed,tts_g,rec_list[0],rec_list[1])
            co.counter(nimg,reel_Angles,personID, level,EXS,EXSType_show,cnt,state,tts_g,rec_list[0],rec_list[1])
            

    return nimg

In [17]:
def pose_estimation_video(filename, EXS, level, EXSType,voice_type):
    #intialize video capture object to read video from built-in camera
    cap = cv2.VideoCapture(filename)
    
    global EXSType_show
    EXSType_show = [None]*5
    global cnt 
    cnt = [0]*5
    global state 
    state = [None]*5
    global EXS_feedback
    EXS_feedback=[None]*5
    global state_fed
    state_fed=[None]*5
    global  tts_g
    tts_g = NBSapi()
    
    #Full screen mode
    WINDOW_NAME = 'Full Integration'
    cv2.namedWindow(WINDOW_NAME, cv2.WND_PROP_FULLSCREEN)
    cv2.setWindowProperty(WINDOW_NAME, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    
    # Capture frames from reel-time camera video
    while cap.isOpened():
        
        #get screen size
        user32 = ctypes.windll.user32
        screen_width, screen_height = user32.GetSystemMetrics(0), user32.GetSystemMetrics(1)
        
        #read video frame by frame
        (ret, frame) = cap.read()
        if ret == True:
            #print number of key-points->17
            print(model.yaml['nkpt'])
            
            #get image scale
            frame_height, frame_width, _ = frame.shape
            scaleWidth = float(screen_width)/float(frame_width)
            scaleHeight = float(screen_height)/float(frame_height)

            if scaleHeight>scaleWidth:
                imgScale = scaleWidth

            else:
                imgScale = scaleHeight

            newX,newY = frame.shape[1]*imgScale, frame.shape[0]*imgScale
            
            #convert the rgb image to grayscale
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            output, frame = run_inference(frame)
            if voice_type != 2: 
                tts_g.SetVoice(voice_type)
                frame = draw_keypoints(output, frame, EXS, level,EXSType,tts_g)
            else:
                frame = draw_keypoints(output, frame, EXS, level,EXSType,voice_type)
            
            #resize frame to fit full screen
            frame = cv2.resize(frame,(int(newX),int(newY)))
            cv2.imshow(WINDOW_NAME, frame)
        else:
            break

        if cv2.waitKey(10) & 0xFF == ord('q'):
            tts_g.Stop()
            break

    cap.release()
    cv2.destroyAllWindows()

In [10]:
def Gui():
    global voice_g
    global hex_color
    voice_g = ""
    tts = NBSapi()
    tts.SetVolume(50)
    tts.SetVoice(1)
    window = tk.Tk()
    window.title("SMART GYM")
    window.geometry("1940x1080+0+0")

    # Load the logo image file
    logo_file = r"D:\Home.png"
    logo_image = Image.open(logo_file)
    logo_photo = ImageTk.PhotoImage(logo_image, master=window)

    # Extract the dominant color from the logo image
    color_thief = ColorThief(logo_file)
    dominant_color = color_thief.get_color(quality=1)

    # Convert the RGB value of the dominant color to a hexadecimal color code
    hex_color = f"#{dominant_color[0]:02x}{dominant_color[1]:02x}{dominant_color[2]:02x}"

    # Create a label to hold the logo image
    logo_label = ttk.Label(window, image=logo_photo)
    logo_label.place(x=500, y=0, relwidth=1, relheight=1)

    # Create a style object to customize ttk widgets
    style = ttk.Style(window)
    style.theme_use("clam")  # Use a modern-looking theme

    # Create a frame to hold the title label
    title_frame = ttk.Frame(window)
    title_frame.pack(fill="x", pady=20)
    title_label = ttk.Label(
        title_frame,
        text="SMART GYM",
        font=('Times New Roman', 30, 'bold'),
        foreground="white",
        padding=(20, 15),
        background=hex_color)
    title_label.pack(fill='x')

    file =  r"D:\countdown.gif"
    
     ####   
    def pose_estimation_window1(file):
        root2 = tk.Toplevel(window)
        root2.title("Pose Estimation")
        root2.geometry("1940x1080+0+0")
        
         # Load the logo image file
        logo_file1 = r"D:\Home.png"
        logo_image1 = Image.open(logo_file1)
        logo_photo1 = ImageTk.PhotoImage(logo_image1, master=root2)

        # Extract the dominant color from the logo image
        color_thief1 = ColorThief(logo_file1)
        dominant_color1 = color_thief1.get_color(quality=1)

        # Convert the RGB value of the dominant color to a hexadecimal color code
        hex_color1 = f"#{dominant_color1[0]:02x}{dominant_color1[1]:02x}{dominant_color1[2]:02x}"

        # Create a label to hold the logo image
        logo_label1 = ttk.Label(root2, image=logo_photo1)
        logo_label1.place(x=500, y=0, relwidth=1, relheight=1)

        # Create a style object to customize ttk widgets
        style = ttk.Style(root2)
        style.theme_use("clam")  # Use a modern-looking theme

        # Create a frame to hold the title label
        title_frame1 = ttk.Frame(root2)
        title_frame1.pack(fill="x", pady=20)

        title_label1 = ttk.Label(
            title_frame1,
            text="SMART GYM",
            font=('Times New Roman', 30, 'bold'),
            foreground="white",
            padding=(20, 15),
            background=hex_color)
        title_label1.pack(fill='x')
        
        #recognition dataset
        biceps_button = ttk.Button(root2, text="Biceps", command=lambda: pose_estimation_window2(file) ,  width=30)
        biceps_button.place(x=250, y=300)

        #recognition training
        shoulder_button = ttk.Button(root2, text="Shoulder", command=lambda: pose_estimation_window3(file) ,  width=30)
        shoulder_button.place(x=250, y=350)

        #recognition testing
        quad_button = ttk.Button(root2, text="Quad", command=lambda: pose_estimation_window4(file) ,  width=30)
        quad_button.place(x=250, y=400)

        #pose estimation
        hamstring_button = ttk.Button(root2, text="Hamstring", command=lambda: pose_estimation_window5(file) ,  width=30)
        hamstring_button.place(x=250, y=450)
    

        back_button = ttk.Button(root2, text="back", command=root2.destroy,  width=30)
        back_button.place(x=250, y=500)
        
        root2.mainloop()
        
        
    ########
    def pose_estimation_window2(file):
        global hex_color

        window2 = tk.Toplevel(window)
        window2.title("SMART GYM")
        window2.geometry("1940x1080+0+0")
        
        logo_file = r"D:\level-1.png"
        logo_image = Image.open(logo_file)
        logo_photo = ImageTk.PhotoImage(logo_image, master=window2)
        
        logo_file2 = r"D:\level-2-icon.png"
        logo_image2 = Image.open(logo_file2)
        logo_photo2 = ImageTk.PhotoImage(logo_image2, master=window2)

        logo_file3 = r"D:\level-icon-15.jpg"
        logo_image3 = Image.open(logo_file3)
        logo_photo3 = ImageTk.PhotoImage(logo_image3, master=window2)
        
        logo_file4 = r"D:\back-icon-png-15.jpg"
        logo_image4 = Image.open(logo_file4)
        logo_photo4 = ImageTk.PhotoImage(logo_image4, master=window2)

        # Create a style object to customize ttk widgets
        style = ttk.Style(window2)
        style.theme_use("clam")  # Use a modern-looking theme

        # Create a frame to hold the title label
        title_frame = ttk.Frame(window2)
        title_frame.pack(fill="x", pady=20)

        title_label = ttk.Label(
            title_frame,
            text="SMART GYM",
            font=('Times New Roman', 30, 'bold'),
            foreground="white",
            padding=(20, 15),
            background=hex_color).pack(fill='x')

        # Create a label to hold the icon image
        logo_label1 = tk.Button(window2,image= logo_photo , command=lambda:biceps_window1(),border=0)
        logo_label1.place(x=400, y=300)
        
        logo_label2 = tk.Button(window2,image= logo_photo2,command=lambda:biceps_window2(),border=0)
        logo_label2.place(x=700, y=300)
        
        logo_label3 = tk.Button(window2,image= logo_photo3,command=lambda:biceps_window3(),border=0)
        logo_label3.place(x=1000, y=300)
        
        logo_label4 = tk.Button(window2,image= logo_photo4,border=0 ,command=window2.destroy,height=200,width=200)
        logo_label4.place(x=100, y=300)
        
        window2.mainloop()
        
    ########
    def pose_estimation_window3(file):
        global hex_color

        window3 = tk.Toplevel(window)
        window3.title("SMART GYM")
        window3.geometry("1940x1080+0+0")
        
        logo_file = r"D:\level-1.png"
        logo_image = Image.open(logo_file)
        logo_photo = ImageTk.PhotoImage(logo_image, master=window3)
        
        logo_file2 = r"D:\level-2-icon.png"
        logo_image2 = Image.open(logo_file2)
        logo_photo2 = ImageTk.PhotoImage(logo_image2, master=window3)

        logo_file3 = r"D:\level-icon-15.jpg"
        logo_image3 = Image.open(logo_file3)
        logo_photo3 = ImageTk.PhotoImage(logo_image3, master=window3)
        
        logo_file4 = r"D:\back-icon-png-15.jpg"
        logo_image4 = Image.open(logo_file4)
        logo_photo4 = ImageTk.PhotoImage(logo_image4, master=window3)

        # Create a style object to customize ttk widgets
        style = ttk.Style(window3)
        style.theme_use("clam")  # Use a modern-looking theme

        # Create a frame to hold the title label
        title_frame = ttk.Frame(window3)
        title_frame.pack(fill="x", pady=20)

        title_label = ttk.Label(
            title_frame,
            text="SMART GYM",
            font=('Times New Roman', 30, 'bold'),
            foreground="white",
            padding=(20, 15),
            background=hex_color).pack(fill='x')

        # Create a label to hold the icon image
        logo_label1 = tk.Button(window3,image= logo_photo , command=lambda:elbow_window(),border=0)
        logo_label1.place(x=400, y=300)
        
        logo_label2 = tk.Button(window3,image= logo_photo2,command=lambda:elbow_window2(),border=0)
        logo_label2.place(x=700, y=300)
    
        
        logo_label4 = tk.Button(window3,image= logo_photo4,border=0 ,command=window3.destroy,height=200,width=200)
        logo_label4.place(x=100, y=300)
        
        window3.mainloop()
        
    ########
    ########
    def pose_estimation_window4(file):
        global hex_color

        window4 = tk.Toplevel(window)
        window4.title("SMART GYM")
        window4.geometry("1940x1080+0+0")
        
        logo_file = r"D:\level-1.png"
        logo_image = Image.open(logo_file)
        logo_photo = ImageTk.PhotoImage(logo_image, master=window4)
        
        logo_file2 = r"D:\level-2-icon.png"
        logo_image2 = Image.open(logo_file2)
        logo_photo2 = ImageTk.PhotoImage(logo_image2, master=window4)

        logo_file3 = r"D:\level-icon-15.jpg"
        logo_image3 = Image.open(logo_file3)
        logo_photo3 = ImageTk.PhotoImage(logo_image3, master=window4)
        
        logo_file4 = r"D:\back-icon-png-15.jpg"
        logo_image4 = Image.open(logo_file4)
        logo_photo4 = ImageTk.PhotoImage(logo_image4, master=window4)

        # Create a style object to customize ttk widgets
        style = ttk.Style(window4)
        style.theme_use("clam")  # Use a modern-looking theme

        # Create a frame to hold the title label
        title_frame = ttk.Frame(window4)
        title_frame.pack(fill="x", pady=20)

        title_label = ttk.Label(
            title_frame,
            text="SMART GYM",
            font=('Times New Roman', 30, 'bold'),
            foreground="white",
            padding=(20, 15),
            background=hex_color).pack(fill='x')

        # Create a label to hold the icon image
        logo_label1 = tk.Button(window4,image= logo_photo , command=lambda: quad_window(),border=0)
        logo_label1.place(x=400, y=300)
        
        logo_label2 = tk.Button(window4,image= logo_photo2,command=lambda: quad_window2(),border=0)
        logo_label2.place(x=700, y=300)
        
        logo_label3 = tk.Button(window4,image= logo_photo3,command=lambda: quad_window3(),border=0)
        logo_label3.place(x=1000, y=300)
        
        logo_label4 = tk.Button(window4,image= logo_photo4,border=0 ,command=window4.destroy,height=200,width=200)
        logo_label4.place(x=100, y=300)
        
        window4.mainloop()
        
    ########
    ########
    def pose_estimation_window5(file):
        global hex_color

        window5 = tk.Toplevel(window)
        window5.title("SMART GYM")
        window5.geometry("1940x1080+0+0")
        
        logo_file = r"D:\level-1.png"
        logo_image = Image.open(logo_file)
        logo_photo = ImageTk.PhotoImage(logo_image, master=window5)
        
        logo_file2 = r"D:\level-2-icon.png"
        logo_image2 = Image.open(logo_file2)
        logo_photo2 = ImageTk.PhotoImage(logo_image2, master=window5)

        logo_file3 = r"D:\level-icon-15.jpg"
        logo_image3 = Image.open(logo_file3)
        logo_photo3 = ImageTk.PhotoImage(logo_image3, master=window5)
        
        logo_file4 = r"D:\back-icon-png-15.jpg"
        logo_image4 = Image.open(logo_file4)
        logo_photo4 = ImageTk.PhotoImage(logo_image4, master=window5)

        # Create a style object to customize ttk widgets
        style = ttk.Style(window5)
        style.theme_use("clam")  # Use a modern-looking theme

        # Create a frame to hold the title label
        title_frame = ttk.Frame(window5)
        title_frame.pack(fill="x", pady=20)

        title_label = ttk.Label(
            title_frame,
            text="SMART GYM",
            font=('Times New Roman', 30, 'bold'),
            foreground="white",
            padding=(20, 15),
            background=hex_color).pack(fill='x')

        # Create a label to hold the icon image
        logo_label1 = tk.Button(window5,image= logo_photo , command=lambda:hamstring_window(),border=0)
        logo_label1.place(x=400, y=300)
    
        
        logo_label4 = tk.Button(window5,image= logo_photo4,border=0 ,command=window5.destroy,height=200,width=200)
        logo_label4.place(x=100, y=300)
        
        window5.mainloop()
    
     ########
    def biceps_window1():
        global hex_color
        bi_window = tk.Toplevel(window)
        bi_window.title("SMART GYM")
        bi_window.geometry("1940x1080+0+0")

        # Create a style object to customize ttk widgets
        style = ttk.Style(bi_window)
        style.theme_use("clam")  # Use a modern-looking theme

        # Create a frame to hold the title label
        title_frame = ttk.Frame(bi_window)
        title_frame.pack(fill="x", pady=20)

        title_label = ttk.Label(
            title_frame,
            text="SMART GYM",
            font=('Times New Roman', 30, 'bold'),
            foreground="white",
            padding=(20, 15),
            background=hex_color).pack(fill='x')
        
        biceps = tk.Button(bi_window,width=500,height=400,border=0)
        biceps.place(x=500, y=200)
        
        back_img = r"D:\back-icon-png-15.jpg"
        back_img = Image.open(back_img)
        back_img = ImageTk.PhotoImage(back_img, master=bi_window)
        
        front_img = r"D:\front1.jpg"
        front_img = Image.open(front_img)
        front_img = ImageTk.PhotoImage(front_img, master=bi_window)

        #biceps animation
        file=r"D:\biceps.gif"
        info = Image.open(file)
        frames = info.n_frames  # gives total number of frames that gif contains

        # creating list of PhotoImage objects for each frames
        im = [tk.PhotoImage(file=file,format=f"gif -index {i}") for i in range(frames)]

        count = 0
        anim = None
        global x
        x=0
        #display animation frames
        def animation(count,x):
            global anim
            if count < frames:
                im2 = im[count]
                biceps.configure(image=im2)
                count += 1
                time.sleep(0.38)
            elif count == frames:
                count=0
            anim = bi_window.after(6,lambda :animation(count,x))
            
        def destroy():
            bi_window.destroy() 

        animation(0,1)
        back = tk.Button(bi_window,image=back_img,border=0 ,command=bi_window.destroy,height=200,width=200)
        back.place(x=200, y=300)
        front = tk.Button(bi_window,image=front_img,border=0 ,command=lambda:[destroy(),start_window("Biceps","Beginner","Biceps Beg")],height=200,width=200)
        front.place(x=1000, y=300)
        bi_window.mainloop()
        
    ########
     ########
    def biceps_window2():
        global hex_color
        bi_window2 = tk.Toplevel(window)
        bi_window2.title("SMART GYM")
        bi_window2.geometry("1940x1080+0+0")

        # Create a style object to customize ttk widgets
        style = ttk.Style(bi_window2)
        style.theme_use("clam")  # Use a modern-looking theme

        # Create a frame to hold the title label
        title_frame = ttk.Frame(bi_window2)
        title_frame.pack(fill="x", pady=20)

        title_label = ttk.Label(
            title_frame,
            text="SMART GYM",
            font=('Times New Roman', 30, 'bold'),
            foreground="white",
            padding=(20, 15),
            background=hex_color).pack(fill='x')
        
        biceps = tk.Button(bi_window2,width=500,height=600,border=0)
        biceps.place(x=500, y=100)
        
        back_img = r"D:\back-icon-png-15.jpg"
        back_img = Image.open(back_img)
        back_img = ImageTk.PhotoImage(back_img, master=bi_window2)
        
        front_img = r"D:\front1.jpg"
        front_img = Image.open(front_img)
        front_img = ImageTk.PhotoImage(front_img, master=bi_window2)

        #biceps animation
        file=r"D:\٢٠٢٣٠٦١١_١٧٢١١٥.gif"
        info = Image.open(file)
        frames = info.n_frames  # gives total number of frames that gif contains

        # creating list of PhotoImage objects for each frames
        im = [tk.PhotoImage(file=file,format=f"gif -index {i}") for i in range(frames)]

        count = 0
        anim = None
        global x
        x=0
        #display animation frames
        def animation(count,x):
            global anim
            if count < frames:
                im2 = im[count]
                biceps.configure(image=im2)
                count += 1
                time.sleep(0.38)
            elif count == frames:
                count=0
            anim = bi_window2.after(6,lambda :animation(count,x))
            
        def destroy():
            bi_window2.destroy() 

        animation(0,1)
        back = tk.Button(bi_window2,image=back_img,border=0 ,command=bi_window2.destroy,height=200,width=200)
        back.place(x=200, y=300)
        front = tk.Button(bi_window2,image=front_img,border=0 ,command=lambda:[destroy(),start_window("Biceps",'Intermediate','Biceps Int')],height=200,width=200)
        front.place(x=1000, y=300)
        bi_window2.mainloop()
        
    ########
    ########
    def biceps_window3():
        global hex_color
        bi_window3 = tk.Toplevel(window)
        bi_window3.title("SMART GYM")
        bi_window3.geometry("1940x1080+0+0")

        # Create a style object to customize ttk widgets
        style = ttk.Style(bi_window3)
        style.theme_use("clam")  # Use a modern-looking theme

        # Create a frame to hold the title label
        title_frame = ttk.Frame(bi_window3)
        title_frame.pack(fill="x", pady=20)

        title_label = ttk.Label(
            title_frame,
            text="SMART GYM",
            font=('Times New Roman', 30, 'bold'),
            foreground="white",
            padding=(20, 15),
            background=hex_color).pack(fill='x')
        
        biceps = tk.Button(bi_window3,width=500,height=600,border=0)
        biceps.place(x=500, y=100)
        
        back_img = r"D:\back-icon-png-15.jpg"
        back_img = Image.open(back_img)
        back_img = ImageTk.PhotoImage(back_img, master=bi_window3)
        
        front_img = r"D:\front1.jpg"
        front_img = Image.open(front_img)
        front_img = ImageTk.PhotoImage(front_img, master=bi_window3)

        #biceps animation
        file=r"D:\٢٠٢٣٠٦١١_١٧١٩٥١.gif"
        info = Image.open(file)
        frames = info.n_frames  # gives total number of frames that gif contains

        # creating list of PhotoImage objects for each frames
        im = [tk.PhotoImage(file=file,format=f"gif -index {i}") for i in range(frames)]

        count = 0
        anim = None
        global x
        x=0
        #display animation frames
        def animation(count,x):
            global anim
            if count < frames:
                im2 = im[count]
                biceps.configure(image=im2)
                count += 1
                time.sleep(0.38)
            elif count == frames:
                count=0
            anim = bi_window3.after(6,lambda :animation(count,x))
            
        def destroy():
            bi_window3.destroy() 

        animation(0,1)
        back = tk.Button(bi_window3,image=back_img,border=0 ,command=bi_window3.destroy,height=200,width=200)
        back.place(x=200, y=300)
        front = tk.Button(bi_window3,image=front_img,border=0 ,command=lambda:[destroy(),start_window("Biceps",'Advanced','Biceps Adv')],height=200,width=200)
        front.place(x=1000, y=300)
        bi_window3.mainloop()
        
    ########
    ########
    def elbow_window():
        global hex_color
        el_window = tk.Toplevel(window)
        el_window.title("SMART GYM")
        el_window.geometry("1940x1080+0+0")

        # Create a style object to customize ttk widgets
        style = ttk.Style(el_window)
        style.theme_use("clam")  # Use a modern-looking theme

        # Create a frame to hold the title label
        title_frame = ttk.Frame(el_window)
        title_frame.pack(fill="x", pady=20)

        title_label = ttk.Label(
            title_frame,
            text="SMART GYM",
            font=('Times New Roman', 30, 'bold'),
            foreground="white",
            padding=(20, 15),
            background=hex_color).pack(fill='x')
        
        elbow = tk.Button(el_window,width=500,height=600,border=0)
        elbow.place(x=500, y=100)
        
        back_img = r"D:\back-icon-png-15.jpg"
        back_img = Image.open(back_img)
        back_img = ImageTk.PhotoImage(back_img, master=el_window)
        
        front_img = r"D:\front1.jpg"
        front_img = Image.open(front_img)
        front_img = ImageTk.PhotoImage(front_img, master=el_window)

        #biceps animation
        file=r"D:\٢٠٢٣٠٦١١_١٧٢١٣١.gif"
        info = Image.open(file)
        frames = info.n_frames  # gives total number of frames that gif contains

        # creating list of PhotoImage objects for each frames
        im = [tk.PhotoImage(file=file,format=f"gif -index {i}") for i in range(frames)]

        count = 0
        anim = None
        global x
        x=0
        #display animation frames
        def animation(count,x):
            global anim
            if count < frames:
                im2 = im[count]
                elbow.configure(image=im2)
                count += 1
                time.sleep(0.38)
            elif count == frames:
                count=0
            anim = el_window.after(6,lambda :animation(count,x))
            
        def destroy():
            el_window.destroy() 

        animation(0,1)
        back = tk.Button(el_window,image=back_img,border=0 ,command=el_window.destroy,height=200,width=200)
        back.place(x=200, y=300)
        front = tk.Button(el_window,image=front_img,border=0 ,command=lambda:[destroy(),start_window('Shoulder','Beginner','Elbow Beg')],height=200,width=200)
        front.place(x=1000, y=300)
        el_window.mainloop()
        
    ########
    ########
    def elbow_window2():
        global hex_color
        el_window2 = tk.Toplevel(window)
        el_window2.title("SMART GYM")
        el_window2.geometry("1940x1080+0+0")

        # Create a style object to customize ttk widgets
        style = ttk.Style(el_window2)
        style.theme_use("clam")  # Use a modern-looking theme

        # Create a frame to hold the title label
        title_frame = ttk.Frame(el_window2)
        title_frame.pack(fill="x", pady=20)

        title_label = ttk.Label(
            title_frame,
            text="SMART GYM",
            font=('Times New Roman', 30, 'bold'),
            foreground="white",
            padding=(20, 15),
            background=hex_color).pack(fill='x')
        
        elbow = tk.Button(el_window2,width=500,height=600,border=0)
        elbow.place(x=500, y=100)
        
        back_img = r"D:\back-icon-png-15.jpg"
        back_img = Image.open(back_img)
        back_img = ImageTk.PhotoImage(back_img, master=el_window2)
        
        front_img = r"D:\front1.jpg"
        front_img = Image.open(front_img)
        front_img = ImageTk.PhotoImage(front_img, master=el_window2)

        #biceps animation
        file=r"D:\٢٠٢٣٠٦١١_١٧٢١٤٧.gif"
        info = Image.open(file)
        frames = info.n_frames  # gives total number of frames that gif contains

        # creating list of PhotoImage objects for each frames
        im = [tk.PhotoImage(file=file,format=f"gif -index {i}") for i in range(frames)]

        count = 0
        anim = None
        global x
        x=0
        #display animation frames
        def animation(count,x):
            global anim
            if count < frames:
                im2 = im[count]
                elbow.configure(image=im2)
                count += 1
                time.sleep(0.38)
            elif count == frames:
                count=0
            anim = el_window2.after(6,lambda :animation(count,x))
            
        def destroy():
            el_window.destroy() 

        animation(0,1)
        back = tk.Button(el_window2,image=back_img,border=0 ,command=el_window2.destroy,height=200,width=200)
        back.place(x=200, y=300)
        front = tk.Button(el_window2,image=front_img,border=0 ,command=lambda:[destroy(),start_window('Shoulder','Intermediate','Elbow Int')],height=200,width=200)
        front.place(x=1000, y=300)
        el_window2.mainloop()
        
    ########
    ########
    def quad_window():
        global hex_color
        q_window = tk.Toplevel(window)
        q_window.title("SMART GYM")
        q_window.geometry("1940x1080+0+0")

        # Create a style object to customize ttk widgets
        style = ttk.Style(q_window)
        style.theme_use("clam")  # Use a modern-looking theme

        # Create a frame to hold the title label
        title_frame = ttk.Frame(q_window)
        title_frame.pack(fill="x", pady=20)

        title_label = ttk.Label(
            title_frame,
            text="SMART GYM",
            font=('Times New Roman', 30, 'bold'),
            foreground="white",
            padding=(20, 15),
            background=hex_color).pack(fill='x')
        
        quad = tk.Button(q_window,width=500,height=600,border=0)
        quad.place(x=500, y=100)
        
        back_img = r"D:\back-icon-png-15.jpg"
        back_img = Image.open(back_img)
        back_img = ImageTk.PhotoImage(back_img, master=q_window)
        
        front_img = r"D:\front1.jpg"
        front_img = Image.open(front_img)
        front_img = ImageTk.PhotoImage(front_img, master=q_window)

        #biceps animation
        file=r"D:\٢٠٢٣٠٦١١_١٧٢٠١٠.gif"
        info = Image.open(file)
        frames = info.n_frames  # gives total number of frames that gif contains

        # creating list of PhotoImage objects for each frames
        im = [tk.PhotoImage(file=file,format=f"gif -index {i}") for i in range(frames)]

        count = 0
        anim = None
        global x
        x=0
        #display animation frames
        def animation(count,x):
            global anim
            if count < frames:
                im2 = im[count]
                quad.configure(image=im2)
                count += 1
                time.sleep(0.38)
            elif count == frames:
                count=0
            anim = q_window.after(6,lambda :animation(count,x))
            
        def destroy():
            q_window.destroy() 

        animation(0,1)
        back = tk.Button(q_window,image=back_img,border=0 ,command=q_window.destroy,height=200,width=200)
        back.place(x=200, y=300)
        front = tk.Button(q_window,image=front_img,border=0 ,command=lambda:[destroy(),start_window('Quad','Beginner','Split Squat Contralateral Load')],height=200,width=200)
        front.place(x=1000, y=300)
        q_window.mainloop()
        
    ########
    ########
    def quad_window2():
        global hex_color
        q_window2 = tk.Toplevel(window)
        q_window2.title("SMART GYM")
        q_window2.geometry("1940x1080+0+0")

        # Create a style object to customize ttk widgets
        style = ttk.Style(q_window2)
        style.theme_use("clam")  # Use a modern-looking theme

        # Create a frame to hold the title label
        title_frame = ttk.Frame(q_window2)
        title_frame.pack(fill="x", pady=20)

        title_label = ttk.Label(
            title_frame,
            text="SMART GYM",
            font=('Times New Roman', 30, 'bold'),
            foreground="white",
            padding=(20, 15),
            background=hex_color).pack(fill='x')
        
        quad = tk.Button(q_window2,width=500,height=600,border=0)
        quad.place(x=500, y=100)
        
        back_img = r"D:\back-icon-png-15.jpg"
        back_img = Image.open(back_img)
        back_img = ImageTk.PhotoImage(back_img, master=q_window2)
        
        front_img = r"D:\front1.jpg"
        front_img = Image.open(front_img)
        front_img = ImageTk.PhotoImage(front_img, master=q_window2)

        #biceps animation
        file=r"D:\٢٠٢٣٠٦١١_١٧٢٠٤٢.gif"
        info = Image.open(file)
        frames = info.n_frames  # gives total number of frames that gif contains

        # creating list of PhotoImage objects for each frames
        im = [tk.PhotoImage(file=file,format=f"gif -index {i}") for i in range(frames)]

        count = 0
        anim = None
        global x
        x=0
        #display animation frames
        def animation(count,x):
            global anim
            if count < frames:
                im2 = im[count]
                quad.configure(image=im2)
                count += 1
                time.sleep(0.38)
            elif count == frames:
                count=0
            anim = q_window2.after(6,lambda :animation(count,x))
            
        def destroy():
            q_window2.destroy() 

        animation(0,1)
        back = tk.Button(q_window2,image=back_img,border=0 ,command=q_window2.destroy,height=200,width=200)
        back.place(x=200, y=300)
        front = tk.Button(q_window2,image=front_img,border=0 ,command=lambda:[destroy(),start_window('Quad','Intermediate','Dumbbell Reverse Lunge Intermediate')],height=200,width=200)
        front.place(x=1000, y=300)
        q_window2.mainloop()
        
    ########
    ########
    def quad_window3():
        global hex_color
        q_window3 = tk.Toplevel(window)
        q_window3.title("SMART GYM")
        q_window3.geometry("1940x1080+0+0")

        # Create a style object to customize ttk widgets
        style = ttk.Style(q_window3)
        style.theme_use("clam")  # Use a modern-looking theme

        # Create a frame to hold the title label
        title_frame = ttk.Frame(q_window3)
        title_frame.pack(fill="x", pady=20)

        title_label = ttk.Label(
            title_frame,
            text="SMART GYM",
            font=('Times New Roman', 30, 'bold'),
            foreground="white",
            padding=(20, 15),
            background=hex_color).pack(fill='x')
        
        quad = tk.Button(q_window3,width=500,height=600,border=0)
        quad.place(x=500, y=100)
        
        back_img = r"D:\back-icon-png-15.jpg"
        back_img = Image.open(back_img)
        back_img = ImageTk.PhotoImage(back_img, master=q_window3)
        
        front_img = r"D:\front1.jpg"
        front_img = Image.open(front_img)
        front_img = ImageTk.PhotoImage(front_img, master=q_window3)

        #biceps animation
        file=r"D:\٢٠٢٣٠٦١١_١٧١٩٠٥.gif"
        info = Image.open(file)
        frames = info.n_frames  # gives total number of frames that gif contains

        # creating list of PhotoImage objects for each frames
        im = [tk.PhotoImage(file=file,format=f"gif -index {i}") for i in range(frames)]

        count = 0
        anim = None
        global x
        x=0
        #display animation frames
        def animation(count,x):
            global anim
            if count < frames:
                im2 = im[count]
                quad.configure(image=im2)
                count += 1
                time.sleep(0.38)
            elif count == frames:
                count=0
            anim = q_window3.after(6,lambda :animation(count,x))
            
        def destroy():
            q_window3.destroy() 

        animation(0,1)
        back = tk.Button(q_window3,image=back_img,border=0 ,command=q_window3.destroy,height=200,width=200)
        back.place(x=200, y=300)
        front = tk.Button(q_window3,image=front_img,border=0 ,command=lambda:[destroy(),start_window('Quad','Advanced','Prisoner Squat')],height=200,width=200)
        front.place(x=1000, y=300)
        q_window3.mainloop()
        
    ########
    ########
    def hamstring_window():
        global hex_color
        h_window = tk.Toplevel(window)
        h_window.title("SMART GYM")
        h_window.geometry("1940x1080+0+0")

        # Create a style object to customize ttk widgets
        style = ttk.Style(h_window)
        style.theme_use("clam")  # Use a modern-looking theme

        # Create a frame to hold the title label
        title_frame = ttk.Frame(h_window)
        title_frame.pack(fill="x", pady=20)

        title_label = ttk.Label(
            title_frame,
            text="SMART GYM",
            font=('Times New Roman', 30, 'bold'),
            foreground="white",
            padding=(20, 15),
            background=hex_color).pack(fill='x')
        
        hamstring = tk.Button(h_window,width=500,height=600,border=0)
        hamstring.place(x=500, y=100)
        
        back_img = r"D:\back-icon-png-15.jpg"
        back_img = Image.open(back_img)
        back_img = ImageTk.PhotoImage(back_img, master=h_window)
        
        front_img = r"D:\front1.jpg"
        front_img = Image.open(front_img)
        front_img = ImageTk.PhotoImage(front_img, master=h_window)

        #biceps animation
        file=r"D:\٢٠٢٣٠٦١١_١٧١٩٢٩.gif"
        info = Image.open(file)
        frames = info.n_frames  # gives total number of frames that gif contains

        # creating list of PhotoImage objects for each frames
        im = [tk.PhotoImage(file=file,format=f"gif -index {i}") for i in range(frames)]

        count = 0
        anim = None
        global x
        x=0
        #display animation frames
        def animation(count,x):
            global anim
            if count < frames:
                im2 = im[count]
                hamstring.configure(image=im2)
                count += 1
                time.sleep(0.38)
            elif count == frames:
                count=0
            anim = h_window.after(6,lambda :animation(count,x))
            
        def destroy():
            h_window.destroy() 

        animation(0,1)
        back = tk.Button(h_window,image=back_img,border=0 ,command=h_window.destroy,height=200,width=200)
        back.place(x=200, y=300)
        front = tk.Button(h_window,image=front_img,border=0 ,command=lambda:[destroy(),start_window('Hamstring','Beginner','Kettlebell Swing')],height=200,width=200)
        front.place(x=1000, y=300)
        h_window.mainloop()
        
    ########
      ########
    def start_window(EXS, level, EXSType):
        global hex_color, voice_type
        path_v,voice_type = "", ""
       
        start_window = tk.Toplevel(window)
        start_window.title("SMART GYM")
        start_window.geometry("1940x1080+0+0")

        # Create a style object to customize ttk widgets
        style = ttk.Style(start_window)
        style.theme_use("clam")  # Use a modern-looking theme

        # Create a frame to hold the title label
        title_frame = ttk.Frame(start_window)
        title_frame.pack(fill="x", pady=20)

        title_label = ttk.Label(
            title_frame,
            text="SMART GYM",
            font=('Times New Roman', 30, 'bold'),
            foreground="white",
            padding=(20, 15),
            background=hex_color).pack(fill='x')
        
        # Path
        path_frame = ttk.Frame(start_window)
        path_frame.pack(pady=10)

        path_label = ttk.Label(path_frame, text="Path", font=('Helvetica', 15))
        path_label.pack(side="left", padx=10)

        path_entry = ttk.Entry(path_frame, width=30, font=('Helvetica', 15))
        path_entry.pack(side="left", padx=10)
        
         # Voice Type
        voice_frame = ttk.Frame(start_window)
        voice_frame.pack(pady=10)

        voice_label = ttk.Label(voice_frame, text="Voice", font=('Helvetica', 15))
        voice_label.pack(side="left", padx=10)

        voice_entry = ttk.Entry(voice_frame, width=30, font=('Helvetica', 15))
        voice_entry.pack(side="left", padx=10)
        
        #count down label
        gif_label = tk.Label(start_window,border=0)
        gif_label.place(x=450, y=100)
        
        #start button
        start_img = r"D:\5200000.png"
        start_img = Image.open(start_img)
        start_img = ImageTk.PhotoImage(start_img, master=start_window)
        
        #back icon
        back_img = r"D:\back-icon-png-15.jpg"
        back_img = Image.open(back_img)
        back_img = ImageTk.PhotoImage(back_img, master=start_window)
        
        

        def get_path():
            path_v = path_entry.get()
            return path_v
        
        def get_voice():
            voice_type = voice_entry.get()
            return voice_type

        #count down animation
        file=r"D:\countdown.gif"
        info = Image.open(file)
        frames = info.n_frames  # gives total number of frames that gif contains

        # creating list of PhotoImage objects for each frames
        im = [tk.PhotoImage(file=file,format=f"gif -index {i}") for i in range(frames)]

        count1 = 0
        anim = None
        global x
        x=0
        #display animation frames
        def animation(count1,x,path_v,level,EXS,EXSType,voice_type):
            global anim
            if count1 < frames:
                im2 = im[count1]
                gif_label.configure(image=im2)
                count1 += 1
                time.sleep(0.38)
            anim = start_window.after(6,lambda :animation(count1,x,int(path_v),level,EXS,EXSType,voice_type))
            if count1 == frames and x<1:
                time.sleep(0.60)
                pose_estimation_video(int(path_v),EXS,level,EXSType,int(voice_type))
                x +=1

        #animation(0,1,get_path(),level,EXS,EXSType,get_voice())
        
         # Create a label to hold the logo image
        start = tk.Button(start_window,image= start_img,command=lambda:animation(1,0,get_path(),level,EXS,EXSType,get_voice()),border=0)
        start.place(x=900, y=500)
        
        #back button
        back = tk.Button(start_window,image=back_img,border=0 ,command=start_window.destroy,height=200,width=200)
        back.place(x=300, y=500)
        
        start_window.mainloop()
        
        
    ########
    
    #recognition dataset
    dataset_button = ttk.Button(window, text="Dataset", command=rec.createDataset ,  width=30)
    dataset_button.place(x=250, y=300)
    
    #recognition training
    train_button = ttk.Button(window, text="Training", command=rec.TrainImages ,  width=30)
    train_button.place(x=250, y=350)

    #recognition testing
    test_button = ttk.Button(window, text="Testing", command=lambda: rec.test(rec.name(), tts) ,  width=30)
    test_button.place(x=250, y=400)

    #pose estimation
    continue_button = ttk.Button(window, text="Continue", command=lambda: pose_estimation_window1(file) ,  width=30)
    continue_button.place(x=250, y=450)

    quit_button = ttk.Button(window, text="Quit", command=window.destroy ,  width=30)
    quit_button.place(x=250, y=500)

    window.mainloop()

In [18]:
Gui()

17
0
[      693.5         326     0.98877         731      286.75     0.99658         699      284.25     0.33667       861.5       314.5     0.97314       732.5      299.75     0.20178         965       497.5     0.36426         675      465.75     0.81982         939       702.5    0.018478       578.5       712.5
     0.22559         854         662    0.027115       695.5       677.5     0.18506         901         713   0.0091629       717.5       703.5    0.023605       810.5         629   0.0039291         682         634   0.0084152         776         632    0.004364         706         636   0.0070381]
17
len1
0
[        493         369     0.99609         525         349     0.99658       456.5         353     0.99658         569         378     0.91602      416.75      384.25     0.79346         657       478.5     0.88574       370.5         514      0.9209       768.5         617     0.27563      370.25         691
     0.43384         757         647     0.19873         